In [47]:
import pandas as pd
import numpy as np
import datetime 
import warnings
warnings.filterwarnings(action = 'ignore')

In [48]:
active_employees=pd.read_excel("../input/active_employees.xlsx")
attrition=pd.read_excel("../input/attrition.xlsx")
training=pd.read_excel("../input/training.xlsx")
incentive=pd.read_excel("../input/sales_incentive.xls",index_col=0)
progression=pd.read_excel("../input/progression.xlsx")
recognition=pd.read_excel("../input/recognition.xlsx",index_col=0)
market_value=pd.read_excel("../input/market_value.xlsx",index_col=0).reset_index()

In [49]:
training.rename(columns={'empno':'Emp.No.', 'empname':'Employee Name', 'org_dept':'Department'},inplace=True)

In [50]:
print(active_employees.shape)
print(attrition.shape)

(3216, 154)
(2647, 87)


In [51]:
incentive.rename(columns={"PERNR":'Emp.No.','BETRG':"incentive"},inplace=True)

In [52]:
training=training[training['sdate']<="2021-03-31"].reset_index()
incentive=incentive[incentive['BEGDA']<="2021-03-31"].reset_index()
recognition=recognition[recognition['Date']<="2021-03-31"].reset_index()
active_employees=active_employees[active_employees['Date of Join']<="2021-03-31"].reset_index()

In [53]:
dat=attrition[['Emp.No.', 'Employee Name',
       'Date of Birth', 'Date of Join', 'Leaving Date', 'Reason', 'Department',
       'Designation', 'Location', 'Exp. in TVSM (Years)',
       'Ext. Exp in years',  'Grade','This Grade from', 
       '2015 - App.Perf','2016 - App.Perf', '2017 - App.Perf', '2018 - App.Perf',
       '2019 - App.Perf', '2020 - App.Perf',
       'Education',  'Mode of Selection', 'Education.1', 'CTC', 'Status.1', 'Gender','Potential Rating','Role','This Role from']]

In [54]:
dat['Resignation Date']=dat['Leaving Date']-pd.DateOffset(months=4)

In [55]:
dat.shape

(2647, 29)

In [56]:
dac=active_employees[[ 'Emp.No.','Employee Name','Date of Birth', 'Date of Join','Department','Designation','Location','Exp.in TVS in Years', 'Ext. Exp in Years',
'Present Grade', 'Grade Effective.Date','PR 20 - 21', 'PR 19 - 20', 'PR 18 - 19','Education','Mode of Selection','Total CTC','Contract Type','Gender','POT 20 - 21','Role','This Role from']]

In [57]:
dac.shape

(2959, 22)

In [58]:
dac['Age']=dac['Date of Birth'].apply(lambda x: (pd.to_datetime("2021-03-31").toordinal()-x.toordinal())/365)

dac['This Role from']=dac['This Role from'].fillna(dac['Date of Join'])
dac['Years_in_Current_Role']=dac['This Role from'].apply(lambda x: (pd.to_datetime("2021-03-31").toordinal()-x.toordinal())/365)

dac['Grade Effective.Date']=dac['Grade Effective.Date'].fillna(dac['Date of Join'])
dac['Years_in_Current_Grade']=dac['Grade Effective.Date'].apply(lambda x: (pd.to_datetime("2021-03-31").toordinal()-x.toordinal())/365)

dac['Attrition']="No"

dac.rename(columns={'Exp.in TVS in Years':"TVS_exp_in_Years", 'Ext. Exp in Years':"External_exp_in_Years","Contract Type":"Type of employment","POT 20 - 21":"Potential Rating",
"Present Grade":"Grade",'Total CTC':"CTC",'Location':'work_location'},inplace=True)

dac['TVS_exp_in_Years']=dac['Date of Join'].apply(lambda x: (pd.to_datetime("2021-03-31").toordinal()-x.toordinal())/365)



In [59]:
doo=dat[dat['Resignation Date']>"2021-03-31"].reset_index(drop=True)
doo['Attrition']="No"
dat=dat[dat['Resignation Date']<="2021-03-31"].reset_index(drop=True)

In [60]:
dat['Age']=((dat['Resignation Date']- dat['Date of Birth'])/ np.timedelta64(1, 'D'))/365

dat['This Role from']=dat['This Role from'].fillna(dac['Date of Join'])
dat['Years_in_Current_Role']=((dat['Resignation Date']- dat['This Role from'])/ np.timedelta64(1, 'D'))/365

dat['This Grade from']=pd.to_datetime(dat['This Grade from'].replace({"00.00.0000":""}))
dat['This Grade from']=dat['This Grade from'].fillna(dat['Date of Join'])
dat['Years_in_Current_Grade']=((dat['Resignation Date']- dat['This Grade from'])/ np.timedelta64(1, 'D'))/365

dat['Education']=dat['Education'].fillna('Education.1')

dat['Attrition']="Yes"

dat.rename(columns={'Exp. in TVSM (Years)':"TVS_exp_in_Years", 'Ext. Exp in years':"External_exp_in_Years","Status.1":"Type of employment",
'2016 - App.Perf':'PR 16 - 17','2017 - App.Perf':'PR 17 - 18', '2018 - App.Perf':'PR 18 - 19', '2019 - App.Perf':"PR 19 - 20",'2020 - App.Perf':'PR 20 - 21','Location':'work_location'},inplace=True)
dat=dat[dat['Reason'].str.contains("Resignation",na=False)]


dat['TVS_exp_in_Years']=((dat['Resignation Date']- dat['Date of Join'])/ np.timedelta64(1, 'D'))/365


In [61]:
doo['Age']=doo['Date of Birth'].apply(lambda x: (pd.to_datetime("2021-03-31").toordinal()-x.toordinal())/365)

doo['This Role from']=doo['This Role from'].fillna(dac['Date of Join'])
doo['Years_in_Current_Role']=doo['This Role from'].apply(lambda x: (pd.to_datetime("2021-03-31").toordinal()-x.toordinal())/365)


doo['This Grade from']=pd.to_datetime(doo['This Grade from'].replace({"00.00.0000":""}))
doo['This Grade from']=doo['This Grade from'].fillna(dat['Date of Join'])
doo['Years_in_Current_Grade']=doo['This Grade from'].apply(lambda x: (pd.to_datetime("2021-03-31").toordinal()-x.toordinal())/365)
doo['Education']=doo['Education'].fillna('Education.1')


doo.rename(columns={'Exp. in TVSM (Years)':"TVS_exp_in_Years", 'Ext. Exp in years':"External_exp_in_Years","Status.1":"Type of employment",
'2016 - App.Perf':'PR 16 - 17','2017 - App.Perf':'PR 17 - 18', '2018 - App.Perf':'PR 18 - 19', '2019 - App.Perf':"PR 19 - 20",'2020 - App.Perf':'PR 20 - 21','Location':'work_location'},inplace=True)
doo=doo[doo['Reason'].str.contains("Resignation",na=False)]


doo['TVS_exp_in_Years']=doo['Date of Join'].apply(lambda x: (pd.to_datetime("2021-03-31").toordinal()-x.toordinal())/365)



In [62]:
df=pd.concat([dac,dat,doo]).reset_index(drop=True)

In [63]:
df

,Emp.No.,Employee Name,Date of Birth,Date of Join,Department,Designation,work_location,TVS_exp_in_Years,External_exp_in_Years,Grade,...,Years_in_Current_Grade,Attrition,Leaving Date,Reason,This Grade from,2015 - App.Perf,PR 16 - 17,PR 17 - 18,Education.1,Resignation Date
0,1,Venu Srinivasan,1952-12-11,1979-05-23,MDO,Chairman & Managing Director,Chennai,41.884932,0.0,S4,...,17.008219,No,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaT
1,279,Harne Vinay Chandrakant,1959-03-17,1982-07-31,NPD,Chief Technology Officer,Hosur Plant,38.693151,0.0,S3,...,16.602740,No,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaT
2,336,K P Chandrashekara,1965-01-26,1985-04-01,R & D (H),Assistant Manager - R & D,Hosur Plant,36.021918,1.6,B1,...,-0.002740,No,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaT
3,340,A Rajan Babu,1952-07-28,1983-09-01,ERM (H),Chief Medical Officer,Hosur Plant,37.605479,4.0,D5,...,16.008219,No,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaT
4,342,M Surianarayanan,1965-04-24,1985-04-01,TQC (H),Deputy Manager - TQC,Hosur Plant,36.021918,0.8,B2,...,0.410959,No,NaT,NaN,NaT,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4392,12817,Nemani Sagar Venkateshwar,1996-08-14,2020-07-22,IB (H),Management Trainee,Hosur Plant,0.690411,1.0,TM,...,0.690411,No,2021-10-18,Resignation - Personal Reasons,2020-07-22,NaN,NaN,NaN,MBA - T1,2021-06-18
4393,13175,Harpreet Singh,1995-10-22,2021-07-07,PRE-OWN (H),Senior Executive,Noida,-0.268493,6.0,A3,...,-0.268493,No,2021-10-18,Resignation - Personal Reasons,2021-07-07,NaN,NaN,NaN,DET,2021-06-18
4394,13250,Yogeshwaran B,2000-02-14,2021-08-11,IT (H),Graduate Engineer Trainee,Hosur Plant,-0.364384,0.0,TM,...,-0.608219,No,2021-10-20,Resignation - Personal Reasons,2021-11-08,NaN,NaN,NaN,GE - T2,2021-06-20
4395,8505,K Arun Kumar,1979-04-03,2012-05-09,Parts (H),Assistant General Manager,Bangalore,8.898630,10.6,C2,...,6.241096,No,2021-10-22,Resignation - Better offer-Com,2015-01-04,EE,ME,ME,MBA,2021-06-22


In [64]:
df['CTC'].replace(0,np.nan,inplace=True)
df['CTC'].fillna(df.groupby('Grade')['CTC'].transform('median'),inplace=True)
df['CTC'].fillna(df['CTC'].median(),inplace=True)


In [65]:
df.loc[df['Department'].isin(['Sales (H)', 'Sales (M)', 'Sales (HP)']),'Median_CTC']=pd.merge(df,market_value,on='Grade',how='left')

In [66]:
df['compa ratio']=df['CTC']/df['Median_CTC']

In [67]:
df['compa ratio'].fillna(1,inplace=True)

In [68]:
tr=training[training.status=="Attended"]
tr['training_days']=(tr.edate-tr.sdate).dt.days
tr['training_days']=tr['training_days'].apply(lambda x: max(x,1))
tr['training_hours']=(tr.time_to.astype(str).str[:2].astype(int)+(tr.time_to.astype(str).str[3:5].astype(int)/60))-tr.time_from.astype(str).str[:2].astype(int)+(tr.time_from.astype(str).str[3:5].astype(int)/60)
tr['training_hours']=tr['training_hours']*tr['training_days']
tr=tr.groupby(['Emp.No.','Employee Name']).aggregate(sum)
df=pd.merge(df,tr,on='Emp.No.',how='left')
df['trainings_attended']=df['training_days'].apply(lambda x: "Yes" if x>=0 else "No" )
df['avg_training_days']=df.training_days/df.TVS_exp_in_Years.apply(lambda x: min((pd.to_datetime("2021-03-31").toordinal()-pd.to_datetime("2015-01-01").toordinal())/365,x))
df['avg_training_days']=df['avg_training_days'].fillna(0)
df['avg_training_hours']=df.training_hours/df.TVS_exp_in_Years.apply(lambda x: min((pd.to_datetime("2021-03-31").toordinal()-pd.to_datetime("2015-01-01").toordinal())/365,x))
df['avg_training_hours']=df['avg_training_hours'].fillna(0)


In [69]:
df.columns

Index(['Emp.No.', 'Employee Name', 'Date of Birth', 'Date of Join',
       'Department', 'Designation', 'work_location', 'TVS_exp_in_Years',
       'External_exp_in_Years', 'Grade', 'Grade Effective.Date', 'PR 20 - 21',
       'PR 19 - 20', 'PR 18 - 19', 'Education', 'Mode of Selection', 'CTC',
       'Type of employment', 'Gender', 'Potential Rating', 'Role',
       'This Role from', 'Age', 'Years_in_Current_Role',
       'Years_in_Current_Grade', 'Attrition', 'Leaving Date', 'Reason',
       'This Grade from', '2015 - App.Perf', 'PR 16 - 17', 'PR 17 - 18',
       'Education.1', 'Resignation Date', 'Median_CTC', 'compa ratio', 'index',
       'training_days', 'training_hours', 'trainings_attended',
       'avg_training_days', 'avg_training_hours'],
      dtype='object')

In [70]:
df=df[df['Type of employment'].isin({"Permanent","Probationery"})]

In [71]:
df['Type of employment'].value_counts()

Permanent       3763
Probationery     278
Name: Type of employment, dtype: int64

In [72]:
df.loc[df['Education'].str.contains('MBA|M.B.A.',na=False),'Education']="MBA"
df.loc[df['Education'].str.contains('DET.',na=False),'Education']="DET"
df.loc[df['Education'].str.contains('PGE',na=False),'Education']="PGE"
df.loc[(df['Education'].str.contains('GE',na=False)) & (df['Education'].str.contains('PGE',na=False)==False),'Education']="GE"
df.loc[df['Education'].str.contains('B.E|B Tech',na=False),'Education']="BE/B.Tech"
df.loc[df['Education'].str.contains('Diploma',na=False),'Education']="Diploma"
df.loc[df['Education'].str.contains('B.COM|B.Com',na=False),'Education']="B.Com"
df.loc[df['Education'].str.contains('M.C.A.|MCA',na=False),'Education']="MCA"
df.loc[df['Education'].str.contains('ICWA',na=False),'Education']="ICWA"
df.loc[df['Education'].str.contains('CA',na=False) & (df['Education'].str.contains('MCA',na=False)==False),'Education']="CA"
df.loc[df['Education'].str.contains('B.SC|B.Sc',na=False),'Education']="B.SC"

In [73]:
df=df.reset_index(drop=True)
df['latest_rating']=0
for i in range(0,len(df)):
    if pd.notnull(df['PR 20 - 21'][i]):
        df['latest_rating'][i]=df['PR 20 - 21'][i]
    elif pd.isnull(df['PR 20 - 21'][i]) and pd.notnull(df['PR 19 - 20'][i]):  
        df['latest_rating'][i]=df['PR 19 - 20'][i] 
    elif pd.isnull(df['PR 19 - 20'][i]) and pd.notnull(df['PR 18 - 19'][i]):  
        df['latest_rating'][i]=df['PR 18 - 19'][i] 
    elif pd.isnull(df['PR 18 - 19'][i]) and pd.notnull(df['PR 17 - 18'][i]):  
        df['latest_rating'][i]=df['PR 17 - 18'][i] 
    else:
        df['latest_rating'][i]=df['PR 16 - 17'][i]

In [74]:
df=df.reset_index(drop=True)
df['previous_rating']=0
for i in range(0,len(df)):
    if pd.notnull(df['PR 20 - 21'][i]):
        df['previous_rating'][i]=df['PR 19 - 20'][i]
    elif pd.isnull(df['PR 20 - 21'][i]) and pd.notnull(df['PR 19 - 20'][i]):  
        df['previous_rating'][i]=df['PR 18 - 19'][i] 
    elif pd.isnull(df['PR 19 - 20'][i]) and pd.notnull(df['PR 18 - 19'][i]):  
        df['previous_rating'][i]=df['PR 17 - 18'][i] 
    elif pd.isnull(df['PR 18 - 19'][i]) and pd.notnull(df['PR 17 - 18'][i]):  
        df['previous_rating'][i]=df['PR 16 - 17'][i] 

In [75]:
rating_scale={'VP':3, 'PM':2, 'EP':5, 'EE':4, 'ME':3, 'NM':1, 'DR':3}

In [76]:
df['latest_rating']=df['latest_rating'].map(rating_scale).fillna(0)
df['previous_rating']=df['previous_rating'].map(rating_scale).fillna(0)

In [77]:
df['rating_delta']=0
for j in range(0,len(df)):
    if df['previous_rating'][j]>0:
        df['rating_delta'][j]=pd.to_numeric(df['latest_rating'][j])-pd.to_numeric(df['previous_rating'][j])
    else: 
        df['rating_delta'][j]=0

In [78]:
potential_rating={'LP':1, 'MP':2, 'HP':3, '-':0,0:0}
df['Potential Rating']=df['Potential Rating'].fillna(0).map(potential_rating)

In [79]:
df['Role']=df['Role'].fillna("no_data")

In [80]:
df['Overall_exp_in_Years']=df.TVS_exp_in_Years + df.External_exp_in_Years

In [81]:
df['CTC_percentile']=df.CTC.rank(pct=True)
df['Age_percentile']=df.Age.rank(pct=True)
df['TVS_exp_in_Years_percentile']=df.TVS_exp_in_Years.rank(pct=True)
df['External_exp_in_Years_percentile']=df.External_exp_in_Years.rank(pct=True)
df['Overall_exp_in_Years_percentile']=df.Overall_exp_in_Years.rank(pct=True)
df['Years_in_Current_Grade_percentile']=df.Years_in_Current_Grade.rank(pct=True)
df['Years_in_Current_Role_percentile']=df.Years_in_Current_Role.rank(pct=True)
df['rating_delta_percentile']=df.rating_delta.rank(pct=True)

In [82]:
progression=progression[['Emp.No.','Progression Grade']]
progression=progression.groupby('Emp.No.').aggregate('count').reset_index()
progression.rename(columns={'Progression Grade':'No of Grade changes'},inplace=True)

df=pd.merge(df,progression,on='Emp.No.',how='left')
df['No of Grade changes'].fillna(0,inplace=True)
df['Grade change per year']=df['No of Grade changes']/df['TVS_exp_in_Years']
df['Grade change per year'].fillna(0,inplace=True)

In [83]:
recognition=recognition.reset_index()
recognition=recognition[['EmployeeCode','Date']]
recognition.rename(columns={'EmployeeCode':'Emp.No.'},inplace=True)
recognition=recognition.groupby('Emp.No.').aggregate('count').reset_index()
recognition.rename(columns={'Date':'Recognitions'},inplace=True)

df=pd.merge(df,recognition,on='Emp.No.',how='left')
df['Recognitions'].fillna(0,inplace=True)

In [84]:
recognition=pd.read_excel("../input/recognition.xlsx",index_col=0).reset_index()
recognition=recognition[['EmployeeCode','Date']]
recognition=recognition.groupby('EmployeeCode').aggregate(max).reset_index()
recognition.rename(columns={'EmployeeCode':'Emp.No.','Date':'Last recognition date'},inplace=True)

df=pd.merge(df,recognition,on='Emp.No.',how='left')


In [85]:
df['Time since last recognition']=0
df.loc[df['Attrition']=="Yes",'Time since last recognition']=((df['Resignation Date']-df['Last recognition date'].astype(np.datetime64))/ np.timedelta64(1, 'D'))/365

from datetime import datetime
df.loc[df['Attrition']=="No",'Time since last recognition']=((datetime.today()-df['Last recognition date'].astype(np.datetime64))/ np.timedelta64(1, 'D'))/365
df.loc[df['Last recognition date'].isna(),'Time since last recognition']=df['TVS_exp_in_Years']

In [86]:
inc=pd.merge(incentive,df[['Emp.No.','Resignation Date']],on='Emp.No.',how='left')
inc3_=inc.sort_values('BEGDA').groupby('Emp.No.').tail(3).reset_index()
inc6_=inc.sort_values('BEGDA').groupby('Emp.No.').tail(6).reset_index()
inc3=inc3_[['Emp.No.','incentive']].groupby('Emp.No.').aggregate('mean').reset_index().rename(columns={'incentive':'last three months avg incentive'})
inc6=inc6_[['Emp.No.','incentive']].groupby('Emp.No.').aggregate('mean').reset_index().rename(columns={'incentive':'last six months avg incentive'})
inc3_sum=inc3_[['Emp.No.','incentive']].groupby('Emp.No.').aggregate('sum').reset_index().rename(columns={'incentive':'last three months incentive'})
inc6_sum=inc6_[['Emp.No.','incentive']].groupby('Emp.No.').aggregate('sum').reset_index().rename(columns={'incentive':'last six months incentive'})

In [87]:
df=pd.merge(df,inc3,on='Emp.No.',how='left')
df=pd.merge(df,inc6,on='Emp.No.',how='left')
df=pd.merge(df,inc3_sum,on='Emp.No.',how='left')
df=pd.merge(df,inc6_sum,on='Emp.No.',how='left')

In [88]:
df['last three months avg incentive'].fillna(0,inplace=True)
df['last six months avg incentive'].fillna(0,inplace=True)

In [89]:
df['last three months incentive utilization']=df['last three months incentive']/df['CTC']
df['last six months incentive utilization']=df['last six months incentive']/df['CTC']

df['last three months incentive utilization'].fillna(0,inplace=True)
df['last six months incentive utilization'].fillna(0,inplace=True)

In [90]:
df=df[df['Department'].isin(['Sales (H)', 'Sales (M)', 'Sales (HP)'])].reset_index(drop=True)

In [91]:
df['Role']=df.Role.replace({'Territory Sales Manager':'Territory Manager (Sales)','Territory Manager - Sales':'Territory Manager (Sales)'})

In [92]:
df['Grade']=df.Grade.map({'TM':0,'A1':1,'A2':2,'A3':3,'B1':4,'B2':5,'B3':6,'C1':7,'C2':8,'C3':9,'D1':10,'D2':11,'D3':12,'D4':13,'D5':14,'E1':15,'E2':16,'E3':17,'E4':18,'E5':19,'E6':20,'E7':21,
'M1':22,'M2':23,'M3':24,'M4':25,'M5':26,'M6':27})

In [93]:
df.to_csv("../input/created/input_sales.csv",index=False)

In [94]:
df.shape

(414, 65)

In [95]:
df.columns

Index(['Emp.No.', 'Employee Name', 'Date of Birth', 'Date of Join',
       'Department', 'Designation', 'work_location', 'TVS_exp_in_Years',
       'External_exp_in_Years', 'Grade', 'Grade Effective.Date', 'PR 20 - 21',
       'PR 19 - 20', 'PR 18 - 19', 'Education', 'Mode of Selection', 'CTC',
       'Type of employment', 'Gender', 'Potential Rating', 'Role',
       'This Role from', 'Age', 'Years_in_Current_Role',
       'Years_in_Current_Grade', 'Attrition', 'Leaving Date', 'Reason',
       'This Grade from', '2015 - App.Perf', 'PR 16 - 17', 'PR 17 - 18',
       'Education.1', 'Resignation Date', 'Median_CTC', 'compa ratio', 'index',
       'training_days', 'training_hours', 'trainings_attended',
       'avg_training_days', 'avg_training_hours', 'latest_rating',
       'previous_rating', 'rating_delta', 'Overall_exp_in_Years',
       'CTC_percentile', 'Age_percentile', 'TVS_exp_in_Years_percentile',
       'External_exp_in_Years_percentile', 'Overall_exp_in_Years_percentile',
      

In [96]:
pd.crosstab(df.latest_rating,df.Attrition,margins=True)

Attrition,No,Yes,All
latest_rating,,,
0.0,28,89,117
1.0,4,0,4
2.0,22,5,27
3.0,163,43,206
4.0,33,11,44
5.0,14,2,16
All,264,150,414


In [97]:
prediction_input=df[df.Attrition=='No']

In [98]:
prediction_input.to_csv("../output/predictions/sales_prediction_input.csv",index=False)

In [99]:
prediction_input = prediction_input.sample(frac=1).reset_index(drop=True) 
    

In [100]:
prediction_input.columns

Index(['Emp.No.', 'Employee Name', 'Date of Birth', 'Date of Join',
       'Department', 'Designation', 'work_location', 'TVS_exp_in_Years',
       'External_exp_in_Years', 'Grade', 'Grade Effective.Date', 'PR 20 - 21',
       'PR 19 - 20', 'PR 18 - 19', 'Education', 'Mode of Selection', 'CTC',
       'Type of employment', 'Gender', 'Potential Rating', 'Role',
       'This Role from', 'Age', 'Years_in_Current_Role',
       'Years_in_Current_Grade', 'Attrition', 'Leaving Date', 'Reason',
       'This Grade from', '2015 - App.Perf', 'PR 16 - 17', 'PR 17 - 18',
       'Education.1', 'Resignation Date', 'Median_CTC', 'compa ratio', 'index',
       'training_days', 'training_hours', 'trainings_attended',
       'avg_training_days', 'avg_training_hours', 'latest_rating',
       'previous_rating', 'rating_delta', 'Overall_exp_in_Years',
       'CTC_percentile', 'Age_percentile', 'TVS_exp_in_Years_percentile',
       'External_exp_in_Years_percentile', 'Overall_exp_in_Years_percentile',
      

In [101]:
df.shape

(414, 65)

In [102]:
df

,Emp.No.,Employee Name,Date of Birth,Date of Join,Department,Designation,work_location,TVS_exp_in_Years,External_exp_in_Years,Grade,...,Grade change per year,Recognitions,Last recognition date,Time since last recognition,last three months avg incentive,last six months avg incentive,last three months incentive,last six months incentive,last three months incentive utilization,last six months incentive utilization
0,735,K Muruganandam,1967-03-05,1986-07-31,Sales (H),Deputy General Manager - Warehouse,Hosur Plant,34.690411,0.8,9,...,0.000000,1.0,2021-08-14,0.470601,0.000000,0.000000,NaN,NaN,0.000000,0.000000
1,760,S Meenakshisundaram,1964-01-04,1987-02-16,Sales (H),Deputy Manager - Network Development,Hosur Plant,34.142466,0.5,5,...,0.000000,0.0,2021-10-18,0.292519,0.000000,0.000000,NaN,NaN,0.000000,0.000000
2,914,V Ramesh,1964-06-19,1990-09-10,Sales (H),Senior Executive - Network Development,Hosur Plant,30.575342,2.9,3,...,0.000000,0.0,2021-10-18,0.292519,0.000000,0.000000,NaN,NaN,0.000000,0.000000
3,925,K Prakash Kamath,1964-11-04,1990-11-05,Sales (H),Senior Manager - Planning,Hosur Plant,30.421918,1.8,7,...,0.000000,0.0,NaN,30.421918,0.000000,0.000000,NaN,NaN,0.000000,0.000000
4,953,Avijit Ghosh,1966-07-01,1992-11-21,Sales (H),Manager - Sales,Ahmedabad,28.375342,7.0,6,...,0.000000,0.0,NaN,28.375342,28733.333333,24410.666667,86200.0,146464.0,0.028959,0.049205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,10855,R Amosh,1996-05-25,2016-07-07,Sales (H),Territory Manager,Tiruppur,4.734247,0.0,3,...,0.422454,0.0,2021-09-01,0.421286,50000.000000,50000.000000,100000.0,100000.0,0.112821,0.112821
410,12064,Tanmay Dey,1995-03-27,2018-07-18,Sales (H),Assistant Manager - Sales,Noida,2.704110,0.0,4,...,0.000000,0.0,NaN,2.704110,75000.000000,75000.000000,75000.0,75000.0,0.077905,0.077905
411,12337,Sohamm Ghoshal,1993-10-17,2019-04-10,Sales (H),Territory Manager,Raipur,1.975342,0.1,4,...,0.000000,1.0,2020-05-01,1.758272,0.000000,0.000000,NaN,NaN,0.000000,0.000000
412,11493,D Raja,1996-09-06,2017-06-08,Sales (H),Territory Executive,Chennai,3.813699,0.0,2,...,0.000000,0.0,NaN,3.813699,75000.000000,75000.000000,75000.0,75000.0,0.157944,0.157944


In [103]:
oot=[7684,9178,11488,11789,12336,12811,12815,12447, 12345, 12784, 13106, 12443,  7285, 12986, 10623, 12213,
       12420, 10855, 12064, 12337, 11493, 12783]
oot=pd.DataFrame(oot).rename(columns={0:"Emp.No."})
oot=pd.merge(oot,attrition[['Emp.No.','Leaving Date']],on='Emp.No.',how='left')
oot['Resignation Date']=oot['Leaving Date']-pd.DateOffset(months=3)
oot['Resignation Date'].fillna("2021-09-01",inplace=True)
oot=oot[['Emp.No.','Resignation Date']]
oot.to_csv("../output/predictions/oot.csv")

In [104]:
df.columns

Index(['Emp.No.', 'Employee Name', 'Date of Birth', 'Date of Join',
       'Department', 'Designation', 'work_location', 'TVS_exp_in_Years',
       'External_exp_in_Years', 'Grade', 'Grade Effective.Date', 'PR 20 - 21',
       'PR 19 - 20', 'PR 18 - 19', 'Education', 'Mode of Selection', 'CTC',
       'Type of employment', 'Gender', 'Potential Rating', 'Role',
       'This Role from', 'Age', 'Years_in_Current_Role',
       'Years_in_Current_Grade', 'Attrition', 'Leaving Date', 'Reason',
       'This Grade from', '2015 - App.Perf', 'PR 16 - 17', 'PR 17 - 18',
       'Education.1', 'Resignation Date', 'Median_CTC', 'compa ratio', 'index',
       'training_days', 'training_hours', 'trainings_attended',
       'avg_training_days', 'avg_training_hours', 'latest_rating',
       'previous_rating', 'rating_delta', 'Overall_exp_in_Years',
       'CTC_percentile', 'Age_percentile', 'TVS_exp_in_Years_percentile',
       'External_exp_in_Years_percentile', 'Overall_exp_in_Years_percentile',
      

In [105]:
doo[doo['Department'].isin(['Sales (H)', 'Sales (M)', 'Sales (HP)'])].reset_index(drop=True)['Emp.No.'].unique()

array([ 7285, 12986, 10623, 12213, 12420, 10855, 12064, 12337, 11493,
       12783])

In [106]:
dr=pd.read_excel("../input/oot.xlsx")

In [107]:
import datetime as dt
dr['Resignation Date'].dt.month.astype(str)+ '/' +dr['Resignation Date'].dt.year.astype(str)

0     9/2021
1     9/2021
2     9/2021
3     9/2021
4     9/2021
5     9/2021
6     9/2021
7     4/2021
8     4/2021
9     4/2021
10    4/2021
11    4/2021
12    5/2021
13    5/2021
14    5/2021
15    5/2021
16    5/2021
17    6/2021
18    6/2021
19    6/2021
20    6/2021
21    7/2021
Name: Resignation Date, dtype: object